In [4]:
#%pip install sentence_transformers==3.0.1

In [1]:
# from sentence_transformers import SentenceTransformer

# query_prompt_name = "s2p_query"

# # for each post, per topic, queries is []
# queries = [
#     "What are some ways to reduce stress?",
#     "What are the benefits of drinking green tea?",
# ]
# # 
# docs = [
#     "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
#     "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
# ]

# # ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)
# query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
# doc_embeddings = model.encode(docs)
# print(query_embeddings.shape, doc_embeddings.shape)


# similarities = model.similarity(query_embeddings, doc_embeddings)
# print(similarities)
# # tensor([[0.8179, 0.2958],
# #         [0.3194, 0.7854]])



In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import re

# Load the pre-trained Sentence Transformer model
mdl_name = 'stella_en_1.5B_v5'

model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)#.cuda()

# Function to generate embeddings
def get_embeddings(text):
    # Generate embeddings
    embedding = model.encode(text)
    return embedding


folder = "llama_data"
res_df = pd.read_csv("../../"+folder+"/results.csv")
res_df = res_df.fillna("")
fea_df = pd.read_csv("../../data/fea_df.csv")

fea_df['description'] = fea_df['description'].apply(
    lambda x: re.sub(r"Must related to [^(]*\(?.*?\)?\.", "", x).strip()
)

/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# example

# loop through each sm_id in res_df.sm_id
# loop through each topic in fea_df.fea
sm_id = "104a474"
topic = "bodyhate"


refer_str = fea_df.loc[fea_df.fea==topic, 'description'].tolist()[0]
returned_str = res_df.loc[res_df.sm_id==sm_id, topic+"_phrases"].tolist()[0]
returned_str_ls = returned_str.split(";") # split returned_str by ";"["i wear baggy clothes to hide what i look like."]#
query_prompt_name = "s2p_query"
query_embeddings = model.encode([refer_str], prompt_name=query_prompt_name)
doc_embeddings = model.encode(returned_str_ls)
# print(query_embeddings.shape, doc_embeddings.shape)
cos_scores = cosine_similarity(query_embeddings, doc_embeddings)[0]
res_str = "; ".join([f"{returned_str_ls[i]} ({cos_scores[i]})" for i in range(len(returned_str_ls))]) + ";"




(1, 1024) (2, 1024)


In [10]:
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os


# Define the function to process each batch of res_df
def process_batch(res_df_batch, fea_df):
    res_df_with_cos_batch = res_df_batch.copy()
    for sm_id in res_df_batch.sm_id.unique():
        for topic in fea_df.fea.unique():
            refer_str = fea_df.loc[fea_df.fea == topic, 'description'].tolist()[0]
            returned_str = res_df_batch.loc[res_df_batch.sm_id == sm_id, f"{topic}_phrases"].tolist()[0]
            
            if returned_str:
                returned_str_ls = returned_str.split(";")
                query_prompt_name = "s2p_query"
                query_embeddings = model.encode([refer_str], prompt_name=query_prompt_name)
                doc_embeddings = model.encode(returned_str_ls)
                cos_scores = cosine_similarity(query_embeddings, doc_embeddings)[0]
                res_str = "; ".join([f"{returned_str_ls[i]} ({cos_scores[i]})" for i in range(len(returned_str_ls))]) + ";"
                res_df_with_cos_batch.loc[res_df_with_cos_batch.sm_id == sm_id, f"{topic}_phrases"] = res_str
    return res_df_with_cos_batch

# Split res_df into 10 batches
res_df_batches = np.array_split(res_df, 10)

# Run each batch in parallel using joblib
results = Parallel(n_jobs=10)(delayed(process_batch)(batch, fea_df) for batch in res_df_batches)

# Combine results back into the main DataFrame
res_df_with_cos = pd.concat(results)


In [11]:
res_df_with_cos.loc[res_df_with_cos.sm_id=="1003i3b",:]

,sm_id,text_w_eos,answer_string,relation,protein,ed,exercise,meal,crave,restrict,...,gain_phrases,calorie_phrases,thinspo_phrases,leanbody_phrases,bodyhate_phrases,feargain_phrases,fearfood_phrases,fearcarb_phrases,nosocialeat_phrases,depressedmood_phrases
1,1003i3b,tw ana body dysmorphia describing body potenti...,"(1) relation: yes, related phrases if any:'my...",1,0.0,1.0,0.0,1.0,0.0,1.0,...,,1400 calorie s a day (0.5580000281333923); bo...,im skinny (0.5418000221252441); lose 35 pound...,lose 35 pounds (0.29010000824928284);,i dont have a flat stomach (0.2605000138282776...,i wont want to recover until im skinny (0.3630...,i want to eat nothing (0.40639999508857727); ...,,it makes me so uncomfortable (0.22120000422000...,


In [12]:
res_df_with_cos.to_csv("../../"+folder+"/results_with_cos_"+mdl_name+".csv", index=False)